**Librerias y configuracion**

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import joblib
import re
import unicodedata
import snowballstemmer
from datetime import datetime

**Funcion para limpiar texto con Steamming**

In [ ]:
stemmer = snowballstemmer.stemmer('spanish')

def limpiar_texto(texto):
    # Pasar a minúsculas
    texto = texto.lower()
    # Eliminar tildes
    texto = ''.join(c for c in unicodedata.normalize('NFD', texto)
                    if unicodedata.category(c) != 'Mn')
    # Eliminar signos de puntuación
    texto = re.sub(r'[^\w\s]', '', texto)
    # Tokenizar separando por espacios
    tokens = texto.split()
    texto_stem = ' '.join([stemmer.stemWord(token) for token in tokens])
    return texto_stem


**Generacion de Log con preguntas desconocias**

In [ ]:
# Función para registrar preguntas desconocidas agrupadas por día
def registrar_desconocida(pregunta_usuario):
    hoy = datetime.now().strftime("%d/%m/%Y")
    log_path = "log_desconocidas.txt"

    try:
        with open(log_path, "r") as f:
            contenido = f.read()
    except FileNotFoundError:
        contenido = ""

    if hoy not in contenido:
        with open(log_path, "a") as f:
            f.write(f"\n📅 {hoy}\n")

    with open(log_path, "a") as f:
        f.write(f"- {pregunta_usuario}\n")


**Cargar y limpiar el dataset**

In [ ]:
# Cargar dataset
dataset = pd.read_csv("datos/data_set.csv", skipinitialspace=True)

# Limpiar preguntas
dataset['Pregunta_limpia'] = dataset['Pregunta'].apply(limpiar_texto)

# Separar preguntas limpias y respuestas
preguntas = dataset['Pregunta_limpia'].values
respuestas = dataset['Respuesta'].values

**Vectorizacion y entrenamiento**

In [ ]:
# Vectorizar
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(preguntas)

# Entrenar modelo
modelo = MultinomialNB()
modelo.fit(X, respuestas)

**Guardar modelo**

In [ ]:
# Guardar modelo y vectorizer
joblib.dump(modelo, "modelo_entrenado.pkl")
joblib.dump(vectorizer, "vectorizer.pkl")

**Funcion de prediccion**

In [ ]:
def obtener_respuesta(pregunta_usuario):
    pregunta_limpia = limpiar_texto(pregunta_usuario)
    pregunta_vectorizada = vectorizer.transform([pregunta_limpia])
    probas = modelo.predict_proba(pregunta_vectorizada)[0]
    indice_max = probas.argmax()
    confianza = probas[indice_max]
    
    rango_confianza = 0.3
    
   # Configuración del cuadro informativo
    ancho = 90
    linea_superior = "╔" + "═" * (ancho - 2) + "╗"
    linea_inferior = "╚" + "═" * (ancho - 2) + "╝"
    linea_separadora = "╠" + "═" * (ancho - 2) + "╣"
    linea_divisoria = "║" + "─" * (ancho - 2) + "║"

    # Contenido
    cabecera = "ANALISIS DEL MODELO SEGUN PROMPT"
    items = [
        f"Cantidad de pruebas en el dataset: {len(preguntas)}",
        f"Cantidad de palabras en el vocabulario: {len(vectorizer.get_feature_names_out())}",
        f"Probabilidad de cada clase: {probas}",
        f"Clase con mayor probabilidad: {confianza}",
        f"Nivel de confianza: {rango_confianza}",
        
        f"Categoría elegida: {modelo.classes_[indice_max]}",
    ]

    # Imprimir cuadro
    print(linea_superior)
    print("║" + cabecera.center(ancho - 2) + "║")
    print(linea_separadora)
    for i, item in enumerate(items):
        print("║ " + item.ljust(ancho - 3) + "║")
        if i < len(items) - 1:
            print(linea_divisoria)
    print(linea_inferior)

    if confianza < rango_confianza:
        registrar_desconocida(pregunta_usuario)
        return "No entendí lo que me preguntaste, ¿me lo podés repetir?"
    return modelo.classes_[indice_max]

**Prueba interactiva en vivo**

In [ ]:
# Interacción en bucle
while True:
    pregunta_usuario = input("Podes preguntarme algo o podes escribir 'salir': ")
    if pregunta_usuario.lower() == "salir":
        break
    print(obtener_respuesta(pregunta_usuario))